# Lab 2: Strands でマルチエージェントワークフローを構築

この Lab では、Strands Agents の ["Agents as Tools"](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/multi-agent/agents-as-tools/) パターンを使用して、シングルエージェントシステムから洗練されたマルチエージェントアーキテクチャへと進化させます。包括的な財務ガイダンスを提供するために、オーケストレーターの下で連携する専門化された財務エージェントを作成します。

この Lab では、専門化されたエージェントが協力して、単一のエージェントの能力を超える複雑な問題を解決する、エンタープライズグレードのエージェント連携を実演します。階層型エージェント設計、ツールラッピングパターン、インテリジェントなリクエストルーティングを学びます。これらは、本番規模の AI システムを構築するために不可欠なスキルです。

![architecture](./images/multi-agent.png)

### システムアーキテクチャ

マルチエージェントシステムは3つのコアコンポーネントで構成されています:

#### 1. Budget Agent（Lab 1 から）
*個人予算、支出分析、財務規律に特化*

| ツール | 説明 | 使用例 |
| --- | --- | --- |
| **calculate_budget_breakdown** | 任意の収入レベルに対する 50/30/20 予算計算 | 月収6000ドルで予算を作成してください |
| **analyze_spending_pattern** | 支出パターン分析とパーソナライズされた推奨 | 月収5000ドルに対して外食費800ドルを分析してください |
| **calculator** | 財務計算と数学的演算 | 予算の20%貯蓄目標を計算してください |
  
#### 2. Financial Analysis Agent
*投資調査、ポートフォリオ管理、市場分析に特化*

| ツール | 説明 | 使用例 |
|------|-------------|------------------|
| **get_stock_analysis** | リアルタイム株式データと包括的分析 | Apple株のパフォーマンスと指標を分析してください |
| **create_diversified_portfolio** | リスクに基づくポートフォリオ推奨と配分 | 10,000ドルで中程度のリスクのポートフォリオを作成してください |
| **compare_stock_performance** | 期間を指定した複数株式のパフォーマンス比較 | Tesla、Apple、Googleの6ヶ月間のパフォーマンスを比較してください |

#### 3. Orchestrator Agent

*専門エージェントを調整し、包括的な応答を統合*

| 機能 | 説明 | 使用例 |
|------------|-------------|------------------|
| **エージェントルーティング** | どの専門家に相談すべきかをインテリジェントに判断 | 予算の質問は Budget Agent へ、投資の質問は Financial Agent へルーティング |
| **マルチエージェント連携** | 複雑なクエリに対して複数エージェントからの洞察を統合 | 「予算作成と投資を支援して」は両方のエージェントを使用 |
| **応答統合** | 複数エージェントの出力から一貫した応答を作成 | 予算分析と投資推奨を組み合わせ |
| **コンテキスト管理** | エージェント間のやり取りを通じて会話フローを維持 | フォローアップの推奨を行う際に以前のアドバイスを記憶 |

In [ ]:
# マルチエージェントシステムに必要な依存関係をインストール
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
# マルチエージェントシステムと株式分析用の依存関係をインポート
from strands import Agent, tool
from strands.models import BedrockModel
from strands.agent.conversation_manager import SummarizingConversationManager
import yfinance as yf
from typing import List
from utils import create_guardrail

### ステップ 1: Budget Agent をツールとしてラップ

In [ ]:
# Lab 1 の Budget Agent をオーケストレーション用ツールとしてラップ
from budget_agent import FinancialReport, budget_agent

@tool
def budget_agent_tool(query: str) -> FinancialReport:
    """予算分析と推奨事項を含む構造化財務レポートを生成します。"""
    try:
        structured_response = budget_agent.structured_output(
            output_model=FinancialReport, prompt=query
        )
        return structured_response
    except Exception as e:
        # エラー時にデフォルトの構造化レスポンスを返す
        return FinancialReport(
            monthly_income=0.0,
            budget_categories=[],
            recommendations=[f"レポート生成エラー: {str(e)}"],
            financial_health_score=1,
        )

### ステップ 2: Financial Analysis Agent を作成

In [ ]:
# 投資に特化した Financial Analysis Agent 用のシステムプロンプトを定義
FINANCIAL_ANALYSIS_PROMPT = """You are a specialized financial analysis agent focused on investment research and portfolio recommendations. Your role is to:

1. Research and analyze stock performance data
2. Create diversified investment portfolios
3. Provide data-driven investment recommendations

You do not provide specific investment advice but rather present analytical data to help users make informed decisions. Always include disclaimers about market risks and the importance of consulting financial advisors."""

In [ ]:
# Financial Analysis Agent 用に Bedrock モデルを設定
bedrock_model = BedrockModel(
    model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    region_name="us-west-2",
    temperature=0.0,  # 財務アドバイスのための決定論的な応答
)

In [ ]:
# yfinance を使用した包括的な株式分析ツール
@tool
def get_stock_analysis(symbol: str) -> str:
    """特定の株式シンボルの包括的な分析を取得します。"""
    try:
        stock = yf.Ticker(symbol)
        info = stock.info
        hist = stock.history(period="1y")

        # 主要指標を計算
        current_price = hist["Close"].iloc[-1]
        year_high = hist["High"].max()
        year_low = hist["Low"].min()
        avg_volume = hist["Volume"].mean()
        price_change = (
            (current_price - hist["Close"].iloc[0]) / hist["Close"].iloc[0]
        ) * 100

        return f"""
📊 {symbol.upper()} の株式分析:
• 現在価格: ${current_price:.2f}
• 52週高値: ${year_high:.2f}
• 52週安値: ${year_low:.2f}
• 年初来変動: {price_change:.2f}%
• 平均日次出来高: {avg_volume:,.0f} 株
• 会社名: {info.get("longName", "N/A")}
• セクター: {info.get("sector", "N/A")}
"""
    except Exception as e:
        return f"❌ {symbol} のデータを取得できませんでした: {str(e)}"

In [ ]:
# リスクに基づいた分散投資ポートフォリオ作成ツール
@tool
def create_diversified_portfolio(risk_level: str, investment_amount: float) -> str:
    """リスクレベル（保守的、中程度、積極的）と投資金額に基づいて分散ポートフォリオを作成します。"""

    portfolios = {
        "conservative": {
            "stocks": ["AAPL", "MSFT", "JNJ", "PG", "KO"],
            "weights": [0.25, 0.25, 0.20, 0.15, 0.15],
            "description": "大型株、配当支払い株に焦点",
        },
        "moderate": {
            "stocks": ["AAPL", "GOOGL", "AMZN", "TSLA", "NVDA"],
            "weights": [0.30, 0.25, 0.20, 0.15, 0.10],
            "description": "成長と安定性のバランスの取れた組み合わせ",
        },
        "aggressive": {
            "stocks": ["TSLA", "NVDA", "AMZN", "GOOGL", "META"],
            "weights": [0.30, 0.25, 0.20, 0.15, 0.10],
            "description": "高成長ポテンシャル株",
        },
    }

    if risk_level.lower() not in portfolios:
        return "❌ リスクレベルは conservative、moderate、aggressive のいずれかである必要があります"

    portfolio = portfolios[risk_level.lower()]

    result = f"""
🎯 {risk_level.upper()} ポートフォリオ推奨（${investment_amount:,.0f}）:
{portfolio["description"]}

ポートフォリオ配分:
"""

    for stock, weight in zip(portfolio["stocks"], portfolio["weights"]):
        allocation = investment_amount * weight
        result += f"• {stock}: {weight * 100:.0f}% (${allocation:,.0f})\n"

    result += "\n⚠️ 免責事項: これは教育目的のみです。投資前に財務アドバイザーに相談してください。"
    return result

In [ ]:
# 期間を指定した複数株式のパフォーマンス比較ツール
@tool
def compare_stock_performance(symbols: List[str], period: str = "1y") -> str:
    """指定期間（1y、6m、3m、1m）にわたる複数株式のパフォーマンスを比較します。"""
    if len(symbols) > 5:
        return "❌ 比較は最大5銘柄に制限してください"

    try:
        performance_data = {}

        for symbol in symbols:
            stock = yf.Ticker(symbol)
            hist = stock.history(period=period)
            if not hist.empty:
                start_price = hist["Close"].iloc[0]
                end_price = hist["Close"].iloc[-1]
                performance = ((end_price - start_price) / start_price) * 100
                performance_data[symbol] = performance

        result = f"📈 株式パフォーマンス比較（{period}）:\n"
        sorted_stocks = sorted(
            performance_data.items(), key=lambda x: x[1], reverse=True
        )

        for stock, performance in sorted_stocks:
            result += f"• {stock}: {performance:+.2f}%\n"

        return result

    except Exception as e:
        return f"❌ 株式比較エラー: {str(e)}"

In [ ]:
# 投資ツールを持つ Financial Analysis Agent を作成
financial_analysis_agent = Agent(
    model=bedrock_model,  # ステップ 1 と同じ bedrock_model を使用
    system_prompt=FINANCIAL_ANALYSIS_PROMPT,
    tools=[get_stock_analysis, create_diversified_portfolio, compare_stock_performance],
)

In [ ]:
# ポートフォリオと株式分析で Financial Analysis Agent をテスト
response = financial_analysis_agent(
    "10,000ドルで中程度のリスクのポートフォリオを作成し、Apple株を分析してください"
)

In [ ]:
%%writefile financial_analysis_agent.py
# Financial Analysis Agent をスタンドアロン Python ファイルにエクスポート

import yfinance as yf
from strands import Agent, tool
from typing import List
from strands.models import BedrockModel

# Financial Analysis Agent システムプロンプト
FINANCIAL_ANALYSIS_PROMPT = """You are a specialized financial analysis agent focused on investment research and portfolio recommendations. Your role is to:

1. Research and analyze stock performance data
2. Create diversified investment portfolios
3. Provide data-driven investment recommendations

You do not provide specific investment advice but rather present analytical data to help users make informed decisions. Always include disclaimers about market risks and the importance of consulting financial advisors."""

bedrock_model = BedrockModel(
    model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    region_name="us-west-2",
    temperature=0.0,  # 財務アドバイスのための決定論的な応答
)


# ツール 1: 株式分析の取得
@tool
def get_stock_analysis(symbol: str) -> str:
    """特定の株式シンボルの包括的な分析を取得します。"""
    try:
        stock = yf.Ticker(symbol)
        info = stock.info
        hist = stock.history(period="1y")

        # 主要指標を計算
        current_price = hist["Close"].iloc[-1]
        year_high = hist["High"].max()
        year_low = hist["Low"].min()
        avg_volume = hist["Volume"].mean()
        price_change = (
            (current_price - hist["Close"].iloc[0]) / hist["Close"].iloc[0]
        ) * 100

        return f"""
📊 {symbol.upper()} の株式分析:
• 現在価格: ${current_price:.2f}
• 52週高値: ${year_high:.2f}
• 52週安値: ${year_low:.2f}
• 年初来変動: {price_change:.2f}%
• 平均日次出来高: {avg_volume:,.0f} 株
• 会社名: {info.get("longName", "N/A")}
• セクター: {info.get("sector", "N/A")}
"""
    except Exception as e:
        return f"❌ {symbol} のデータを取得できませんでした: {str(e)}"


# ツール 2: 分散ポートフォリオの作成
@tool
def create_diversified_portfolio(risk_level: str, investment_amount: float) -> str:
    """リスクレベル（保守的、中程度、積極的）と投資金額に基づいて分散ポートフォリオを作成します。"""

    portfolios = {
        "conservative": {
            "stocks": ["AAPL", "MSFT", "JNJ", "PG", "KO"],
            "weights": [0.25, 0.25, 0.20, 0.15, 0.15],
            "description": "大型株、配当支払い株に焦点",
        },
        "moderate": {
            "stocks": ["AAPL", "GOOGL", "AMZN", "TSLA", "NVDA"],
            "weights": [0.30, 0.25, 0.20, 0.15, 0.10],
            "description": "成長と安定性のバランスの取れた組み合わせ",
        },
        "aggressive": {
            "stocks": ["TSLA", "NVDA", "AMZN", "GOOGL", "META"],
            "weights": [0.30, 0.25, 0.20, 0.15, 0.10],
            "description": "高成長ポテンシャル株",
        },
    }

    if risk_level.lower() not in portfolios:
        return "❌ リスクレベルは conservative、moderate、aggressive のいずれかである必要があります"

    portfolio = portfolios[risk_level.lower()]

    result = f"""
🎯 {risk_level.upper()} ポートフォリオ推奨（${investment_amount:,.0f}）:
{portfolio["description"]}

ポートフォリオ配分:
"""

    for stock, weight in zip(portfolio["stocks"], portfolio["weights"]):
        allocation = investment_amount * weight
        result += f"• {stock}: {weight * 100:.0f}% (${allocation:,.0f})\n"

    result += "\n⚠️ 免責事項: これは教育目的のみです。投資前に財務アドバイザーに相談してください。"
    return result


# ツール 3: 株式パフォーマンスの比較
@tool
def compare_stock_performance(symbols: List[str], period: str = "1y") -> str:
    """指定期間（1y、6m、3m、1m）にわたる複数株式のパフォーマンスを比較します。"""
    if len(symbols) > 5:
        return "❌ 比較は最大5銘柄に制限してください"

    try:
        performance_data = {}

        for symbol in symbols:
            stock = yf.Ticker(symbol)
            hist = stock.history(period=period)
            if not hist.empty:
                start_price = hist["Close"].iloc[0]
                end_price = hist["Close"].iloc[-1]
                performance = ((end_price - start_price) / start_price) * 100
                performance_data[symbol] = performance

        result = f"📈 株式パフォーマンス比較（{period}）:\n"
        sorted_stocks = sorted(
            performance_data.items(), key=lambda x: x[1], reverse=True
        )

        for stock, performance in sorted_stocks:
            result += f"• {stock}: {performance:+.2f}%\n"

        return result

    except Exception as e:
        return f"❌ 株式比較エラー: {str(e)}"


# Financial Analysis Agent を作成
financial_analysis_agent = Agent(
    model=bedrock_model,  # ステップ 1 と同じ bedrock_model を使用
    system_prompt=FINANCIAL_ANALYSIS_PROMPT,
    tools=[get_stock_analysis, create_diversified_portfolio, compare_stock_performance],
    callback_handler=None,
)

if __name__ == "__main__":
    # Financial Analysis Agent をテスト
    response = financial_analysis_agent(
        "10,000ドルで中程度のリスクのポートフォリオを作成し、Apple株を分析してください"
    )
    print(response)

In [ ]:
# スタンドアロンの Financial Analysis Agent をテスト
!python financial_analysis_agent.py 

### ステップ 3: Financial Analysis Agent をツールとしてラップ

In [ ]:
# Financial Analysis Agent をオーケストレーター用ツールとしてラップ
from financial_analysis_agent import financial_analysis_agent


@tool
def financial_analysis_agent_tool(query: str) -> str:
    """株式調査、ポートフォリオ作成、パフォーマンス比較を含む投資分析クエリを処理します。"""
    try:
        response = financial_analysis_agent(query)
        return str(response)
    except Exception as e:
        return f"❌ 財務分析エラー: {str(e)}"

### ステップ 4: Orchestrator Agent を作成

In [ ]:
# マルチエージェント調整用のオーケストレーターシステムプロンプトを定義
ORCHESTRATOR_PROMPT = """You are a comprehensive financial advisor orchestrator that coordinates between specialized financial agents to provide complete financial guidance. 

Your specialized agents are:
1. **budget_agent**: Handles budgeting, spending analysis, savings recommendations, and expense tracking
2. **financial_analysis_agent_tool**: Handles investment analysis, stock research, portfolio creation, and performance comparisons

Guidelines for using your agents:
- Use **budget_agent** for questions about: budgets, spending habits, expense tracking, savings goals, debt management
- Use **financial_analysis_agent_tool** for questions about: stocks, investments, portfolios, market analysis, investment recommendations
- You can use both agents together for comprehensive financial planning
- Always provide a cohesive summary that combines insights from multiple agents when applicable
- Maintain a helpful, professional tone and include appropriate disclaimers about financial advice

When a user asks a question:
1. Determine which agent(s) are most appropriate
2. Call the relevant agent(s) with focused queries
3. Synthesize the responses into a coherent, comprehensive answer
4. Provide actionable next steps when possible"""

In [ ]:
# オーケストレーターエージェント用のガードレールを作成
guardrail_id, guardrail_arn = create_guardrail()

In [ ]:
# ガードレール付きでオーケストレーター用 Bedrock モデルを設定
bedrock_model = BedrockModel(
    model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    region_name="us-west-2",
    temperature=0.0,  # 財務アドバイスのための決定論的な応答
    guardrail_id=guardrail_id,  # Bedrock ガードレール ID
    guardrail_version="DRAFT",  # ガードレールバージョン
    guardrail_trace="enabled",
)

In [ ]:
# オーケストレーター用の会話マネージャーを設定
conversation_manager = SummarizingConversationManager(
    summary_ratio=0.3,  # コンテキスト削減が必要な場合、メッセージの30%を要約
    preserve_recent_messages=5,  # 常に直近5件のメッセージを保持
)

In [ ]:
# 両方の専門エージェントツールを持つオーケストレーターエージェントを作成
orchestrator_agent = Agent(
    model=bedrock_model,
    system_prompt=ORCHESTRATOR_PROMPT,
    tools=[budget_agent_tool, financial_analysis_agent_tool],
    conversation_manager=conversation_manager,  # 会話マネージャーを関連付け
)

In [ ]:
# 複雑なマルチエージェントクエリでオーケストレーターをテスト
response = orchestrator_agent("テスラとAppleの株を比較して、月収4000ドルで2000ドルの投資が可能かどうか教えてください。",)

In [ ]:
%%writefile main.py
# 完全なマルチエージェントシステムを main.py にエクスポート

from strands import Agent, tool
from strands.models import BedrockModel
from strands.agent.conversation_manager import SummarizingConversationManager

from budget_agent import FinancialReport, budget_agent
from financial_analysis_agent import financial_analysis_agent

from utils import get_guardrail_id

# マルチエージェント調整用のオーケストレーターシステムプロンプト
ORCHESTRATOR_PROMPT = """You are a comprehensive financial advisor orchestrator that coordinates between specialized financial agents to provide complete financial guidance. 

Your specialized agents are:
1. **budget_agent**: Handles budgeting, spending analysis, savings recommendations, and expense tracking
2. **financial_analysis_agent_tool**: Handles investment analysis, stock research, portfolio creation, and performance comparisons

Guidelines for using your agents:
- Use **budget_agent** for questions about: budgets, spending habits, expense tracking, savings goals, debt management
- Use **financial_analysis_agent_tool** for questions about: stocks, investments, portfolios, market analysis, investment recommendations
- You can use both agents together for comprehensive financial planning
- Always provide a cohesive summary that combines insights from multiple agents when applicable
- Maintain a helpful, professional tone and include appropriate disclaimers about financial advice

When a user asks a question:
1. Determine which agent(s) are most appropriate
2. Call the relevant agent(s) with focused queries
3. Synthesize the responses into a coherent, comprehensive answer
4. Provide actionable next steps when possible"""

# コンテキストを維持するための会話管理を追加
conversation_manager = SummarizingConversationManager(
    summary_ratio=0.3,  # コンテキスト削減が必要な場合、メッセージの30%を要約
    preserve_recent_messages=5,  # 常に直近5件のメッセージを保持
)

# 以前の設定を継続
bedrock_model = BedrockModel(
    model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    region_name="us-west-2",
    temperature=0.0,  # 財務アドバイスのための決定論的な応答
    guardrail_id=get_guardrail_id(),
    guardrail_version="DRAFT",
    guardrail_trace="enabled",
)


@tool
def budget_agent_tool(query: str) -> FinancialReport:
    """予算分析と推奨事項を含む構造化財務レポートを生成します。"""
    try:
        structured_response = budget_agent.structured_output(
            output_model=FinancialReport, prompt=query
        )
        return structured_response
    except Exception as e:
        # エラー時にデフォルトの構造化レスポンスを返す
        return FinancialReport(
            monthly_income=0.0,
            budget_categories=[],
            recommendations=[f"レポート生成エラー: {str(e)}"],
            financial_health_score=1,
        )


# Financial Analysis Agent をツールとしてラップ
@tool
def financial_analysis_agent_tool(query: str) -> str:
    """株式調査、ポートフォリオ作成、パフォーマンス比較を含む投資分析クエリを処理します。"""
    try:
        response = financial_analysis_agent(query)
        return str(response)
    except Exception as e:
        return f"❌ 財務分析エラー: {str(e)}"


# オーケストレーターエージェントを作成
orchestrator_agent = Agent(
    model=bedrock_model,
    system_prompt=ORCHESTRATOR_PROMPT,
    tools=[budget_agent_tool, financial_analysis_agent_tool],
    conversation_manager=conversation_manager,
)

if __name__ == "__main__":
    # オーケストレーターエージェントを作成してテスト
    orchestrator_agent = Agent(
        model=bedrock_model,
        system_prompt=ORCHESTRATOR_PROMPT,
        tools=[budget_agent_tool, financial_analysis_agent_tool],
    )

    response = orchestrator_agent("月収6000ドルで月500ドルの投資を始めたいです。予算を作成し、投資ポートフォリオを提案してください。")

In [ ]:
# 完全なマルチエージェントオーケストレーターシステムをテスト
!python main.py 